## Configuración

In [1]:
# Cargamos paquetes
library(dplyr)
library(abind)
library(loadeR)
library(transformeR)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: rJava

Loading required package: loadeR.java

Java version 23x amd64 by N/A detected

NetCDF Java Library v4.6.0-SNAPSHOT (23 Apr 2015) loaded and ready

Loading required package: climate4R.UDG

climate4R.UDG version 0.2.6 (2023-06-26) is loaded

Please use 'citation("climate4R.UDG")' to cite this package.

loadeR version 1.8.1 (2023-06-22) is loaded


Get the latest stable version (1.8.2) using <devtools::install_github(c('SantanderMetGroup/climate4R.UDG','SantanderMetGroup/loadeR'))>

Please use 'citation("loadeR")' to cite this package.




    _______   ____  ___________________  __  ________ 
   / ___/ /  / /  |/  / __  /_  __/ __/ / / / / __  / 
  / /  / /  / / /|_/ / /_/ / / / / __/ / /_/ / /_/_/  
 / /__/ /__/ / /  / / __  / / / / /__ /___  / / \ \ 
 \___/____/_/_/  /_/_/ /_/ /_/  \___/    /_/\/   \_\ 
 
      github.com/SantanderMetGroup/climate4R



transformeR version 2.2.2 (2023-10-26) is loaded


Get the latest stable version (2.2.3) using <devtools::install_github('SantanderMetGroup/transformeR')>

Please see 'citation("transformeR")' to cite this package.



In [2]:
source("../load_bc_functions.R")

In [3]:
# Region de estudio

lon = c(-10, 5)
lat = c(35, 44)

## Carga de datos del modelo SEAS5

In [4]:
# HINDCAST
anios = 1981:2016
meses_ini = c("12")  # mes de inicialización

# Función generalizada para cargar los datos por mes y año
cargar_dato = function(anio, mes_ini) {
    
    yyyymm = paste0(anio, mes_ini)

    ruta = paste0(
        "/lustre/gmeteo/PTICLIMA/DATA/SEASONAL/",
        "seasonal-original-single-levels/medcof/hindcast/tasmin/ecmwf/51/", yyyymm, "/",
        "seasonal-original-single-levels_medcof_hindcast_tasmin_ecmwf_51_", yyyymm, ".ncml"
    )

    data_aux = loadGridData(dataset = ruta,
                            var = "tasmin",
                            lonLim = lon,
                            latLim = lat,
                            season = c(01, 02, 03, 04, 05, 06)) %>% suppressMessages %>% suppressWarnings

    # FILTRADO DE FECHAS (Hasta 15 de Junio)
    # Extraemos las fechas cargadas
    fechas = as.POSIXct(data_aux$Dates$start, tz = "GMT")
    
    # Extraemos mes y dia
    meses_num = as.numeric(format(fechas, "%m"))
    dias_num  = as.numeric(format(fechas, "%d"))
    
    # Identificamos los índices que queremos mantener:
    # (Meses menores a 6) O (Mes igual a 6 Y dia <= 15)
    ind_validos = which(meses_num < 6 | (meses_num == 6 & dias_num <= 15))
    
    # Recortamos el grid usando subsetDimension
    data_aux = subsetDimension(data_aux, dimension = "time", indices = ind_validos)

    # Agregamos
    data_aux = aggregateGrid(data_aux, aggr.d = list(FUN = "mean", na.rm = TRUE)) %>% suppressMessages %>% suppressWarnings

    return(data_aux)
}

# Creo una lista donde cada elemento es la salida para un mes de inicialización diferente
hindcast = lapply(meses_ini, function(mes) {
    lapply(anios, function(anio) cargar_dato(anio, mes))})

# Nombro los elementos por mes
names(hindcast) = paste0("mes_", meses_ini)

# Asigno nombre a cada grid con los leadtime
var_hcst_1 = hindcast[["mes_12"]]

# Combinamos los grids en la dimensión temporal
var_hcst_1_grid = bindGrid(var_hcst_1, dimension = "time")

In [5]:
# FORECAST
anios = 2017:2021
meses_ini = c("12")  # mes de inicialización

# Función generalizada para cargar los datos por mes y año
cargar_dato = function(anio, mes_ini) {
    
    yyyymm = paste0(anio, mes_ini)

    ruta = paste0(
        "/lustre/gmeteo/PTICLIMA/DATA/SEASONAL/",
        "seasonal-original-single-levels/medcof/forecast/tasmin/ecmwf/51/", yyyymm, "/",
        "seasonal-original-single-levels_medcof_forecast_tasmin_ecmwf_51_", yyyymm, ".ncml"
    )

    data_aux = loadGridData(dataset = ruta,
                            var = "tasmin",
                            lonLim = lon,
                            latLim = lat,
                            season = c(01, 02, 03, 04, 05, 06)) %>% suppressMessages %>% suppressWarnings

    # FILTRADO DE FECHAS (Hasta 15 de Junio)
    # Extraemos las fechas cargadas
    fechas = as.POSIXct(data_aux$Dates$start, tz = "GMT")
    
    # Extraemos mes y dia
    meses_num = as.numeric(format(fechas, "%m"))
    dias_num  = as.numeric(format(fechas, "%d"))
    
    # Identificamos los índices que queremos mantener:
    # (Meses menores a 6) O (Mes igual a 6 Y dia <= 15)
    ind_validos = which(meses_num < 6 | (meses_num == 6 & dias_num <= 15))
    
    # Recortamos el grid usando subsetDimension
    data_aux = subsetDimension(data_aux, dimension = "time", indices = ind_validos)

    # Agregamos
    data_aux = aggregateGrid(data_aux, aggr.d = list(FUN = "mean", na.rm = TRUE)) %>% suppressMessages %>% suppressWarnings

    return(data_aux)
}

# Creo una lista donde cada elemento es la salida para un mes de inicialización diferente
forecast = lapply(meses_ini, function(mes) {
    lapply(anios, function(anio) cargar_dato(anio, mes))})

# Nombro los elementos por mes
names(forecast) = paste0("mes_", meses_ini)

# Asigno nombre a cada grid con los leadtime
var_fcst_1 = forecast[["mes_12"]]

# Combinamos los grids en la dimensión temporal
var_fcst_1_grid = bindGrid(var_fcst_1, dimension = "time")

# Selecciono los primeros 25 miembros
var_fcst_1_members = subsetGrid(var_fcst_1_grid, members = 1:25)

# Combinamos los grids de hindcast y forecast
var_seas5_1 = bindGrid(var_hcst_1_grid, var_fcst_1_members, dimension = "time")

saveRDS(var_seas5_1, "tasmin_nfd_seas5_1.rds")

## Carga de datos de ERA5

In [7]:
# Define años y meses
anios = 1982:2022
meses = sprintf("%02d", c(01, 02, 03, 04, 05, 06))

# Función para construir la ruta y cargar los datos
cargar_dato = function(anio, mes) {
    yyyy = paste0(anio)
    yyyymm = paste0(anio, mes)
    ruta = paste0(
        "/lustre/gmeteo/PTICLIMA/DATA/REANALYSIS/ERA5-Land/data/Iberia/day/t2mn/", yyyy, "/",
        "t2mn_ERA5-Land_", yyyymm, ".nc"
    )
    
    # Carga el dataset
    data_aux = loadGridData(dataset = ruta,
                            var = "t2mn",
                            lonLim = lon,
                            latLim = lat) %>% suppressMessages %>% suppressWarnings

    # FILTRADO DE FECHAS (Hasta 15 de Junio)
    # Extraemos las fechas cargadas
    fechas = as.POSIXct(data_aux$Dates$start, tz = "GMT")
    
    # Extraemos mes y dia
    meses_num = as.numeric(format(fechas, "%m"))
    dias_num  = as.numeric(format(fechas, "%d"))
    
    # Identificamos los índices que queremos mantener:
    # (Meses menores a 6) O (Mes igual a 6 Y dia <= 15)
    ind_validos = which(meses_num < 6 | (meses_num == 6 & dias_num <= 15))
    
    # Recortamos el grid usando subsetDimension
    data_aux = subsetDimension(data_aux, dimension = "time", indices = ind_validos)

    # Agregamos
    data_aux = aggregateGrid(data_aux, aggr.d = list(FUN = "mean", na.rm = TRUE)) %>% suppressMessages %>% suppressWarnings

    return(data_aux)
}

# Generamos la tabla de combinaciones lógica
combinaciones_list = lapply(anios, function(y) {
    data.frame(anio = y, mes = meses)})
combinaciones = do.call(rbind, combinaciones_list)

# Aplicar la función a cada combinación
var_era5_1_data = lapply(1:nrow(combinaciones), function(i) {
    cargar_dato(combinaciones$anio[i], combinaciones$mes[i])})

# Combinamos los grids en la dimensión temporal
var_era5_1_time = bindGrid(var_era5_1_data, dimension = "time")

# Upscaling de la resolución de las observaciones
var_era5_1_ups = interpGrid(var_era5_1_time,
                            new.coordinates = getGrid(var_seas5_1),
                            method = "bilinear") %>% suppressMessages %>% suppressWarnings

# Pasamos las observaciones de Kelvin a Celsius
var_era5_1 = gridArithmetics(var_era5_1_ups, 273.15, operator = "-")
attr(var_era5_1$Variable, "units") = "degC"

saveRDS(var_era5_1, "tasmin_nfd_era5_1.rds")

## Bias Correction

In [ ]:
folds_list = lapply(1982:2022, function(x) x)

var_seas5_1_bc = biasCorrection_RM(
    y = var_era5_1,
    x = var_seas5_1,
    newdata = var_seas5_1,
    method = "eqm",
    extrapolation = "constant",
    cross.val = 'kfold',
    folds = folds_list) %>% suppressMessages %>% suppressWarnings

saveRDS(var_seas5_1_bc, "tasmin_nfd_seas5_1_bc.rds")